In [1]:
#按行读取数据
file = open('/ml/sougou/user_tag_query.2W.TRAIN')
file_lines = file.readlines()
file_linenum = len(file_lines)
file.close()

#age:    0：未知年龄; 1：0-18岁; 2：19-23岁; 3：24-30岁; 4：31-40岁; 5：41-50岁; 6： 51-999岁
#gender: 0：未知1：男性2：女性
#edu:    0：未知学历; 1：博士; 2：硕士; 3：大学生; 4：高中; 5：初中; 6：小学

#通过正则表达式说去数据
import re
pattern = '([0-9A-Z]*)\s([0-6]{1})\s([0-2]{1})\s([0-6]{1})\s(.*)'

#strIDs = [1] * file_linenum
IDlist = []
Agelist = []
Genderlist = []
Edulist = []
Datalist = []

nRealSize = 0

for i in range( 0 , file_linenum ):
    m = re.search( pattern , file_lines[i] )
    if m == None:
        print( '[ERROR]bad lines:' , i )
        break
    else:
        nRealSize += 1
        IDlist.append(m.group(1))
        Agelist.append(m.group(2))
        Genderlist.append(m.group(3))
        Edulist.append(m.group(4))
        Datalist.append(m.group(5))
    
print( "读取原始数据%d条" % (nRealSize) )

#wwws = mycut( Datalist[1000] )
#print ','.join(wwws)
#print (len(wwws))

读取原始数据20000条


In [2]:
import io
import os
import jieba.posseg as pseg

stopwords = [line.strip().decode('utf-8', 'ignore') for line in open('/ml/sougou/stopwords.txt').readlines()]
print( "读取停用词%d条" % (len(stopwords)) )

def mycut( strCnText ):
    words = pseg.cut(strCnText)
    nword = []
    for w in words:
        #print( w.word + " " + w.flag )
        #去除一些不关心的词:[c  连词][p  介词][q  量词][t  时间词][m  量词][eng  英文字母]
        if((w.flag != 'c' and w.flag != 'p' and w.flag != 'q' and w.flag != 't'and w.flag != 'm'and w.flag != 'eng') and len(w.word)>1):
            #通过停用词库去除一些不需要考虑的词汇
            if w.word not in stopwords:
                nword.append(w.word)
            #else:
                #print( "STOP:" + w.word + " " + w.flag )
        #else:
            #print( "CX  :" + w.word + " " + w.flag )
            
    return nword

#开始对数据进行读取
org_cut_data_file = '/ml/sougou/orgcut.dat'
def create_org_cut_data():
    if ( os.path.exists(org_cut_data_file) == True ):
        cget = raw_input("File exist,cover it?(y / n)")
        if ( cget == 'y' ):
            os.remove( org_cut_data_file )
        else:
            return
        
    #创建文件
    print( "开始创建文件....." )
    fp = open( org_cut_data_file , 'w' )
    for i in range( 0 , nRealSize ):
    #for i in range( 0 , 10 ):
        if ( i % 100 == 0 ):
            print( "%d / %d" % ( i , nRealSize ) )
            fp.flush()
        fp.write( ','.join(mycut(Datalist[i])).encode('utf') + '\n' )
    fp.close()
        
#加载数据
def load_org_cut_data():
    if ( os.path.exists(org_cut_data_file) == False ):
        create_org_cut_data()
    lines = [line.strip().decode('utf', 'ignore') for line in open(org_cut_data_file).readlines()]
    orgcutdata = []
    for line in lines:
        orgcutdata.append( line.split(',') )
    return orgcutdata

#对数据进行二次处理，一方面是将数据改为空格间隔，另一方面是可以对数据进行进一步的处理
def org_to_final_cut_data(org_cut_data):
    cutdata = []
    for words in org_cut_data:
        #这里可以对words进行进一步的调整
        
        
        cutdata.append( ' '.join(words) )
        #cutdata.append( ' '.join(set(words)) )
        
    return cutdata
        
    
#create_org_cut_data()
final_cut_data = org_to_final_cut_data( load_org_cut_data() )
print( "读取切分后数据%d条" % len(final_cut_data) )

读取停用词2612条
读取切分后数据20000条


In [3]:
#部分数据清除？
for i in range( 0 , nRealSize )[::-1]:
    if Genderlist[i] == '0':
        IDlist.pop(i)
        Genderlist.pop(i)
        final_cut_data.pop(i)
    
        

In [4]:
#切分数据

import scipy as sp  
import numpy as np  
from sklearn.datasets import load_files  
from sklearn.cross_validation import train_test_split  
from sklearn.feature_extraction.text import  TfidfVectorizer
  
doc_terms_train, doc_terms_test, y_train, y_test = train_test_split(final_cut_data, Genderlist, test_size = 0.3)  

print( "train:%d , test:%d" % ( len(doc_terms_train) , len(doc_terms_test) ) )

train:13703 , test:5873


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
#转换

from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
vectorizer=CountVectorizer()

#该类会统计每个词语的tf-idf权值
transformer=TfidfTransformer()

#第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵
x_train = transformer.fit_transform(vectorizer.fit_transform(doc_terms_train))
x_test = transformer.transform(vectorizer.transform(doc_terms_test))

#x_train = vectorizer.fit_transform(doc_terms_train)
#x_test = vectorizer.transform(doc_terms_test)

print( vectorizer.fit_transform(doc_terms_train)[0].todense() )
print( doc_terms_train[0] )
print( x_train[0].todense() )
print( y_train[0] )

x2_train = []
for n in x_train:
    x2_train.append( n.todense() )


[[0 0 0 ..., 0 0 0]]
表格 求和 平均数 大哥 花生 杨洋 择偶 标准 微信 服务 吸粉 活动 策划 吸粉 投票 活动 策划 定制 益力 软糖 酸奶 敷脸 好处 南京 媒体 运营 招聘 南京 智联 招聘 试吃 名单 公布 星座 零食 乐蜂 聚美 优品 九门 现场 良品 铺子 内容 运营 杨洋 粉刺 吃货 图片 中奖 葡萄 寒性 水果 苹果 年轻 撑腰 点击 购买 有趣 星座 零食 玛璐托 香蕉 巧克力 蛋糕 芜湖 智联 招聘 技能 宏亚 月光 舞曲 礼盒 青云 南京 媒体 运营 双鱼座 短信 营销 开心 茶馆 南京 皮炎 主题 威化 图片 微信 公众 分享 补水 效果 护肤品 定制 养乐多 软糖 微信 公众 活动 策划 方案 吃货 晒图 活动 玛璐托 活动 策划 方案 主题 运营 招聘 捷森 麦片 微信 公众 吸粉 成功 案例 茶馆 开心 茶馆 南京 简笔画 摩羯座 白羊座 惩罚 男生 方法 大全 超狠 星座 零食 蜡笔小新 羊羹 月饼 点击 购买 图标 素材 夏黑 葡萄 提子 鼻子 旁边 红血丝 聊天 晚安 校园生活 威化 专员 岗位职责 黑头 小窍门 简单 龙门 飞爪 好吃 引导 购买 吃货 技巧 韩美 釜山 朋友圈 互动 话题 羊羹 葡萄 好吃 点击 购买 图标 肯德基 推出 防晒霜 卡萨 京都 原味 奶茶 智联 招聘 写给 会员 食代 零食 专题 送礼 误认为 有钱 体验 淘宝 系统 微信 公众 吸粉 活动 分享 南京 运营 招聘 零食 主题 南京 朗睿 科技 葡萄 引导 扫描 二维码 广告语 写给 会员 一段话 黑头 药物 治疗 免费 试吃 报名 方法 主题 嘉宝 引导 扫描 二维码 搞笑 广告语 好看 人字 微信 公众 服务 吸粉 活动 星七 粒粒 威化 摩羯座 美伦多 榴莲 益力 软糖 团圆 江苏 地图 吃货 会员 活动 策划 方案 微信 公众 活动 互动 话题 变得 传播 美食 定个 目标 客户 管理 品牌 点击 购买 试吃 统一 留言 巧克力 韩国 春雨 面膜 蜡笔小新 羊羹 表格 加减乘除 混合 运算 白色 恋人
[[ 0.  0.  0. ...,  0.  0.  0.]]
2


In [140]:
#二分

import numpy as np
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import precision_score, recall_score, accuracy_score

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
predictions = classifier.predict(x_test)

nRight = 0
for i in range( 0 , len(doc_terms_test) ):
    if ( predictions[i] == y_test[i] ):
        nRight += 1
print( nRight * 100 / len(doc_terms_test) )
print('准确率:%s' % accuracy_score(y_test, predictions))
#scores = cross_val_score(classifier, x_train, y_train, cv=5)
#print(' 准确率:',np.mean(scores), scores)

79
准确率:0.792780521028


In [6]:
#高斯朴素贝叶斯  
import numpy as np 
from sklearn.naive_bayes import GaussianNB  
clf = GaussianNB().fit(x_train.todense(), y_train )
predictions = clf.predict( x_test )

nRight = 0
for i in range( 0 , len(doc_terms_test) ):
    if ( predictions[i] == y_test[i] ):
        nRight += 1
print( nRight * 100 / len(doc_terms_test) )
print('准确率:%s' % accuracy_score(y_test, predictions))

MemoryError: 